In [ ]:
import numpy as np
import json
import os
from NeuralNetStructure.imagetools import ImageTools
import math, cv2

In [ ]:
%pwd

In [ ]:
from preprocessor import Preprocessor
data = Preprocessor("planes", "planesnet.json")
X, y, X_test, y_test = data.preprocess(0.75, dims="2D")


In [ ]:
from NeuralNetStructure import layer, loss, accuracy, activation, optimizers, nnet, pool, dropout



In [ ]:
model = nnet.Model("2D", resolution=(100, 100))

model.add(layer.Layer_Conv(input_channels=1, num_filters=3, weight_regularizer_L2=0, bias_regularizer_L2=0), "CONV")
model.add(activation.Activation_ReLU(), "CONV")
model.add(pool.Pool(pool_dims=(2, 2), stride=(2, 2)), "CONV")
model.add(layer.Layer_Conv(input_channels=3, num_filters=3, weight_regularizer_L2=0, bias_regularizer_L2=0), "CONV")
model.add(activation.Activation_ReLU(), "CONV")
model.add(pool.Pool(pool_dims=(2, 2), stride=(2, 2)), "CONV")
model.add(layer.Layer_Flatten(), "CONV")

In [ ]:
model.conv_dims
dense_feature_num = int(model.conv_dims[-1][0] * model.conv_dims[-1][1] * model.conv_dims[-1][2])
print(model.conv_dims, dense_feature_num)

In [ ]:
model.add(layer.Layer_Dense(dense_feature_num, 100, 
    weight_regularizer_L2=1e-4, bias_regularizer_L2=1e-4), 'DENSE')
model.add(activation.Activation_ReLU(), "DENSE")
model.add(dropout.Layer_Dropout(0.2), 'DENSE')

model.add(layer.Layer_Dense(100, 1, 
weight_regularizer_L2=1e-4, bias_regularizer_L2=1e-4), 'DENSE')

model.add(activation.Activation_Sigmoid(), 'DENSE')

model.set(
    loss=loss.Loss_BinaryCrossentropy(),
    optimizer=optimizers.Optimizer_Adam(learning_rate=0.001, decay=2e-7),
    accuracy=accuracy.Accuracy_Binary()
)

model.finalize()


In [ ]:
model.save_parameters("Memory\\planes\\Parameters")

In [ ]:
model.save(os.path.join("VisualCortex", "planesnet.nucleus"))

In [ ]:
model.train(X[:1000], y[:1000], epochs=50, batch_size=32, print_every=10, validation_data=(X_test[:20], y_test[:20]))
